In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
import matplotlib.pyplot as plt
%matplotlib inline

d:\users\pooria\anaconda3\envs\myonlyenv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
gamma = 0.99
def discounted(r):
    res = np.zeros_like(r)
    temp = 0
    for i in reversed(range(0,r.size)):
        temp = temp*gamma + r[i]
        res[i] = temp
    return res

In [3]:
discounted(np.arange(20))

array([166, 168, 169, 168, 167, 165, 161, 157, 152, 145, 137, 129, 119,
       108,  96,  83,  68,  53,  36,  19])

In [4]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
class Agent():
    def __init__(self, lr, s_size, a_size, h_size):
        self.state_in = tf.placeholder(shape=[None, s_size],dtype=tf.float32)
        self.hidden = slim.fully_connected(self.state_in,h_size,biases_initializer=None,activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(self.hidden,a_size,activation_fn=tf.nn.softmax)
        
        self.chosen_action = tf.argmax(self.output,1)
        
        
        self.reward_holder = tf.placeholder(shape=[None],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None],dtype=tf.int32)
        
        self.indexes = tf.range(0,tf.shape(self.output)[0])*tf.shape(self.output)[1] + self.action_holder
        
        self.responsible_weights = tf.gather(tf.reshape(self.output,shape=[-1]),self.indexes)
        
        self.loss = - tf.reduce_mean(tf.log(self.responsible_weights)*self.reward_holder)
        
        
        t_vars = tf.trainable_variables()
        
        self.gradiant_holders = []
        
        for idx,par in enumerate(t_vars):
            placeholder = tf.placeholder(tf.float32,name=str(idx)+"_holder")
            self.gradiant_holders.append(placeholder)
#         the next lines result should go in to the place holders we made and then those will be used to update
        self.gradiant = tf.gradients(self.loss,t_vars)
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate=lr)
        
        self.update_paramaters = self.optimizer.apply_gradients(zip(self.gradiant_holders,t_vars))
        
        

In [9]:
def tryAgents(total_episodes = 5000, max_per = 999, lr=1e-2, h_size=8, freq = 5, afterWhichEpisode = None, paintEnv=True, showReward=True):
    tf.reset_default_graph()
    myAgent = Agent(lr, 4, 2, h_size)

    init = tf.global_variables_initializer()
    
    if afterWhichEpisode == None :
        afterWhichEpisode = int((total_episodes*3)/4)
    show = False
    
    with tf.Session() as sess :
        sess.run(init)
        gradient_buffer = sess.run(tf.trainable_variables())
        i = 0
        total_rewards =[]
        total_length = []


        for idx,grad in enumerate(gradient_buffer):
            gradient_buffer[idx] = grad * 0
        while i < total_episodes :
            if paintEnv and i > afterWhichEpisode :
                show = True
            history = []
            rewardTillNow = 0
            s = env.reset()
            if show :
                env.render()
            for j in range (total_episodes):
                a_dist = sess.run(myAgent.output,feed_dict={myAgent.state_in:[s]})
                a = np.random.choice(a_dist[0],p=a_dist[0])
                a = np.argmax(a_dist == a)



                s1,r,d,_ = env.step(a)
                if show :
                    env.render()

                history.append([s,a,r,s1])

                rewardTillNow += r

                s = s1

                if d is True :
                    history = np.array(history)
                    history[:,2] = discounted(history[:,2])
                    feed_dict = {myAgent.reward_holder:history[:,2],myAgent.action_holder:history[:,1],myAgent.state_in:np.vstack(history[:,0])}
                    grads = sess.run(myAgent.gradiant,feed_dict=feed_dict)
                    for idx,grad in enumerate(grads):
                        gradient_buffer[idx] += grad

                    if i%freq == 0 and i != 0:
                        feed_dict = dict(zip(myAgent.gradiant_holders,gradient_buffer))
                        _ = sess.run(myAgent.update_paramaters,feed_dict=feed_dict)
                        for idx,grad in enumerate(gradient_buffer):
                            gradient_buffer[idx] = grad*0
                    total_rewards.append(rewardTillNow)
                    total_length.append(j)
                    break

            i+=1
            if i%100 == 0 and i != 0 and showReward :
                print(str(i//100)+" : the average reward for the last 100 episodes : "+str(np.mean(total_rewards[-100:])))
        return total_rewards,total_length

In [10]:
# if you want to see whats happening in the env turn the paintEnv arg to True but it'll take more time to train
tr,tl = tryAgents(total_episodes=4000,paintEnv=False)

d:\users\pooria\anaconda3\envs\myonlyenv\lib\site-packages\tensorflow\python\ops\gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


1 : the average reward for the last 100 episodes : 30.84
2 : the average reward for the last 100 episodes : 33.46
3 : the average reward for the last 100 episodes : 41.86
4 : the average reward for the last 100 episodes : 49.08
5 : the average reward for the last 100 episodes : 72.36
6 : the average reward for the last 100 episodes : 87.38
7 : the average reward for the last 100 episodes : 136.21
8 : the average reward for the last 100 episodes : 166.32
9 : the average reward for the last 100 episodes : 188.63
10 : the average reward for the last 100 episodes : 194.66
11 : the average reward for the last 100 episodes : 189.29
12 : the average reward for the last 100 episodes : 193.32
13 : the average reward for the last 100 episodes : 198.92
14 : the average reward for the last 100 episodes : 198.54
15 : the average reward for the last 100 episodes : 198.48
16 : the average reward for the last 100 episodes : 194.51
17 : the average reward for the last 100 episodes : 195.14
18 : the ave

In [12]:
print('the longest episode of actions is : '+str(max(tl)))

the longest episode of actions is : 199
